In [31]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

red_data_loc = r"C:\Users\marcu\OneDrive\Documents\GitHub\MekRelLab\Lab 8 og 11 - lysets hastighed\rødt lys.xlsx"
red_xls = pd.ExcelFile(red_data_loc)

# Indlæs alle ark i excelfilen som numpy arrays
red_data = [np.array(pd.read_excel(red_xls, sheet_name=sheet)) for sheet in red_xls.sheet_names]
# Indlæs hvert ark som en variabel for sig
red_1 = red_data[0]
red_2 = red_data[1]
red_3 = red_data[2]
red_4 = red_data[3]
red_5 = red_data[4]

# Afstandene er værdien i første række af anden søjle i arket (ganget med 2 for at tage højde for afstanden tilbage til sensoren)
# Enhed [m]
red_1_d = red_1[0][1]*2
red_2_d = red_2[0][1]*2
red_3_d = red_3[0][1]*2
red_4_d = red_4[0][1]*2
red_5_d = red_5[0][1]*2

# Tiderne er de første værdier af hver række i arket ganget med 10**(-9) for at få dem i sekunder
# Enhed [s]
red_1_t = red_1[:,0]*10**(-9)
red_2_t = red_2[:,0]*10**(-9)
red_3_t = red_3[:,0]*10**(-9)
red_4_t = red_4[:,0]*10**(-9)
red_5_t = red_5[:,0]*10**(-9)

# En funktion til at beregne lysets gennemsnitlige hastighed ud fra tiderne og afstandene
def c(times, distances):
    return np.mean(distances/times)

print("d: {}m ({}m), c: {:.2e}".format(red_1_d/2, red_1_d, c(red_1_t, red_1_d)))
print("d: {}m ({}m), c: {:.2e}".format(red_2_d/2, red_2_d, c(red_2_t, red_2_d)))
print("d: {}m ({}m), c: {:.2e}".format(red_3_d/2, red_3_d, c(red_3_t, red_3_d)))
print("d: {}m ({}m), c: {:.2e}".format(red_4_d/2, red_4_d, c(red_4_t, red_4_d)))
print("d: {}m ({}m), c: {:.2e}".format(red_5_d/2, red_5_d, c(red_5_t, red_5_d)))




d: 0.675m (1.35m), c: 3.53e+08
d: 2.86m (5.72m), c: 3.47e+08
d: 0.755m (1.51m), c: 2.75e+08
d: 1.965m (3.93m), c: 3.84e+08
d: 2.467m (4.934m), c: 4.02e+08


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

red_data_loc = r"C:\Users\marcu\OneDrive\Documents\GitHub\MekRelLab\Lab 8 og 11 - lysets hastighed\rødt lys.xlsx"
red_xls = pd.ExcelFile(red_data_loc)

# Indlæs alle ark i excelfilen som numpy arrays
red_data = [np.array(pd.read_excel(red_xls, sheet_name=sheet)) for sheet in red_xls.sheet_names]
# Indlæs hvert ark som en variabel for sig
red_1 = red_data[0]
red_2 = red_data[1]
red_3 = red_data[2]
red_4 = red_data[3]
red_5 = red_data[4]

# Afstandene er værdien i første række af anden søjle i arket (ganget med 2 for at tage højde for afstanden tilbage til sensoren)
# Enhed [m]
red_1_d = red_1[0][1]*2
red_2_d = red_2[0][1]*2
red_3_d = red_3[0][1]*2
red_4_d = red_4[0][1]*2
red_5_d = red_5[0][1]*2

# Tiderne er de første værdier af hver række i arket ganget med 10**(-9) for at få dem i sekunder
# Enhed [s]
red_1_t = red_1[:,0]*10**(-9)
red_2_t = red_2[:,0]*10**(-9)
red_3_t = red_3[:,0]*10**(-9)
red_4_t = red_4[:,0]*10**(-9)
red_5_t = red_5[:,0]*10**(-9)

# Gør det samme for de andre farver (blå og grøn)
blue_data_loc = r"C:\Users\marcu\OneDrive\Documents\GitHub\MekRelLab\Lab 8 og 11 - lysets hastighed\blåt lys.xlsx"
blue_xls = pd.ExcelFile(blue_data_loc)

blue_data = [np.array(pd.read_excel(blue_xls, sheet_name=sheet)) for sheet in blue_xls.sheet_names]

blue_1 = blue_data[0]
blue_2 = blue_data[1]
blue_3 = blue_data[2]
blue_4 = blue_data[3]
blue_5 = blue_data[4]

blue_1_d = blue_1[0][1]*2
blue_2_d = blue_2[0][1]*2
blue_3_d = blue_3[0][1]*2
blue_4_d = blue_4[0][1]*2
blue_5_d = blue_5[0][1]*2

blue_1_t = blue_1[:,0]*10**(-9)
blue_2_t = blue_2[:,0]*10**(-9)
blue_3_t = blue_3[:,0]*10**(-9)
blue_4_t = blue_4[:,0]*10**(-9)
blue_5_t = blue_5[:,0]*10**(-9)

green_data_loc = r"C:\Users\marcu\OneDrive\Documents\GitHub\MekRelLab\Lab 8 og 11 - lysets hastighed\grønt lys.xlsx"
green_xls = pd.ExcelFile(green_data_loc)

green_data = [np.array(pd.read_excel(green_xls, sheet_name=sheet)) for sheet in green_xls.sheet_names]

green_1 = green_data[0]
green_2 = green_data[1]
green_3 = green_data[2]
green_4 = green_data[3]
green_5 = green_data[4]

green_1_d = green_1[0][1]*2
green_2_d = green_2[0][1]*2
green_3_d = green_3[0][1]*2
green_4_d = green_4[0][1]*2
green_5_d = green_5[0][1]*2

green_1_t = green_1[:,0]*10**(-9)
green_2_t = green_2[:,0]*10**(-9)
green_3_t = green_3[:,0]*10**(-9)
green_4_t = green_4[:,0]*10**(-9)
green_5_t = green_5[:,0]*10**(-9)

# En funktion til at beregne lysets gennemsnitlige hastighed ud fra tiderne og afstandene
def c(times, distances):
    return np.mean(distances/times)

print(c(red_1_t, red_1_d))

# Usikkerhed for afstandsmålingerne
# Enhed [m]
d_unc = 0.005

# Usikkerhed for tidsmålingerne
# Enhed [s]
t_unc = 0.1*10**(-9)

# Usikkerhed for hastigheden (vha. ophobningsloven)
# Enhed [m/s]




FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\marcu\\OneDrive\\Documents\\GitHub\\MekRelLab\\Lab 8 og 11 - lysets hastighed\\blåt lys.xlsx'